In [168]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem.snowball import SnowballStemmer
import string
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import Word2Vec


import imblearn
# oversampling a multi-class classification dataset
from imblearn.over_sampling import SMOTE

In [169]:
print(gensim.__version__)

4.0.1


In [170]:
df=pd.read_csv('E:\\private\\Intern\\Ratings-Project\\Ratings Project\\RatingData.csv')

In [171]:
df.iloc[[14725],[2]] =1

In [172]:
df=df.dropna()

In [173]:
df.isna().sum()

category    0
review      0
rating      0
dtype: int64

In [174]:
df[df.rating<1]

,category,review,rating
1278,laptop,Its too cool laptop.. easy to use and simple t...,0
9507,laptop,Dell inspiron 15 7560 laptop are best laptop f...,0


In [175]:
df=df.drop([1278,9507])

In [176]:
df[df.rating<1]

,category,review,rating


In [177]:
stop_words=set(stopwords.words('english'))

In [178]:
#Lets do  stemming.It will remove ing,ed and all from the words
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

In [179]:
#data preprocessing/data cleaning
def cleanDocument(df_data):
    df_data=df_data.str.replace('[^a-zA-Z ]', '')
    df_data=df_data.str.replace(r'\s+',' ')
    df_data=df_data.apply(lambda x:' '.join(term for term in x.split() if term not in stop_words))
    df_data =df_data.apply(stemming)
    df_data=df_data.apply(lambda x:' '.join(term for term in x.split() if len(term) > 3 ))
    df_data=df_data.str.lower()
    return df_data
    

In [180]:
df['review']=cleanDocument(df['review'])

In [181]:
tf_vec=TfidfVectorizer(max_features=5000,ngram_range=(1,2))
features=tf_vec.fit_transform(df['review'])
x=features

In [182]:
Y= df.iloc[:,-1]

In [183]:
tf_vec.get_feature_names()

['abil',
 'about',
 'absolut',
 'acceler',
 'accept',
 'accer',
 'access',
 'accessori',
 'accident',
 'accident damag',
 'accompani',
 'accord',
 'accord price',
 'account',
 'accur',
 'accuraci',
 'acer',
 'acer aspir',
 'acer compani',
 'acer good',
 'acer laptop',
 'acer product',
 'acer servic',
 'achiev',
 'across',
 'action',
 'activ',
 'actual',
 'adapt',
 'adaptor',
 'addict',
 'addit',
 'addon',
 'address',
 'adequ',
 'adjust',
 'administr',
 'adob',
 'adopt',
 'advanc',
 'advanc featur',
 'advantag',
 'advantag laptop',
 'advertis',
 'advic',
 'advis',
 'aesthet',
 'affect',
 'afford',
 'afford laptop',
 'afford price',
 'after',
 'after month',
 'after year',
 'again',
 'agent',
 'agre',
 'ahead',
 'alienwar',
 'alli',
 'allow',
 'allround',
 'almost',
 'almost everi',
 'almost hour',
 'almost year',
 'along',
 'alot',
 'alreadi',
 'also',
 'also also',
 'also amaz',
 'also attract',
 'also avail',
 'also awesom',
 'also batteri',
 'also best',
 'also better',
 'also camera

In [184]:
#train and predict
x_train,x_test,y_train,y_test=train_test_split(x,Y,test_size=0.20,random_state=42,shuffle=True)

In [185]:
mn_nb=MultinomialNB()
mn_nb.fit(x_train,y_train)
y_pred=mn_nb.predict(x_test)
print('Multinomial NB metrics::::::::::::')
print('accuracy score::::::::::::::',accuracy_score(y_test,y_pred))
print('classification report ::::::::::::::\n',classification_report(y_test,y_pred))
print('confusion matrix ::::::::::::::\n',confusion_matrix(y_test,y_pred))


Multinomial NB metrics::::::::::::
accuracy score:::::::::::::: 0.41219903691813803
classification report ::::::::::::::
               precision    recall  f1-score   support

           1       0.61      0.64      0.62       336
           2       0.00      0.00      0.00       195
           3       0.30      0.05      0.08       688
           4       0.39      0.62      0.48       989
           5       0.39      0.47      0.43       907

    accuracy                           0.41      3115
   macro avg       0.34      0.35      0.32      3115
weighted avg       0.37      0.41      0.36      3115

confusion matrix ::::::::::::::
 [[214   0  11  78  33]
 [ 69   0  19  87  20]
 [ 30   0  33 359 266]
 [ 21   0  22 611 335]
 [ 19   0  24 438 426]]


C:\Users\STS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\STS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\STS\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [186]:
from sklearn.ensemble import RandomForestClassifier
rf_=RandomForestClassifier()
rf_.fit(x_train,y_train)
y_pred=rf_.predict(x_test)
print('RandomForestClassifier metrics::::::::::::')
print('accuracy score::::::::::::::',accuracy_score(y_test,y_pred))
print('classification report ::::::::::::::\n',classification_report(y_test,y_pred))
print('confusion matrix ::::::::::::::\n',confusion_matrix(y_test,y_pred))

RandomForestClassifier metrics::::::::::::
accuracy score:::::::::::::: 0.4134831460674157
classification report ::::::::::::::
               precision    recall  f1-score   support

           1       0.56      0.67      0.61       336
           2       0.18      0.01      0.02       195
           3       0.33      0.09      0.15       688
           4       0.40      0.58      0.47       989
           5       0.39      0.47      0.43       907

    accuracy                           0.41      3115
   macro avg       0.37      0.36      0.34      3115
weighted avg       0.39      0.41      0.37      3115

confusion matrix ::::::::::::::
 [[226   3  14  58  35]
 [ 82   2  14  73  24]
 [ 49   0  65 316 258]
 [ 23   5  48 571 342]
 [ 21   1  57 404 424]]


In [187]:
from sklearn.neighbors import KNeighborsClassifier
kneighbor=KNeighborsClassifier()
kneighbor.fit(x_train,y_train)
y_pred=kneighbor.predict(x_test)
print('KNeighborsClassifier metrics::::::::::::')
print('accuracy score::::::::::::::',accuracy_score(y_test,y_pred))
print('classification report ::::::::::::::\n',classification_report(y_test,y_pred))
print('confusion matrix ::::::::::::::\n',confusion_matrix(y_test,y_pred))

KNeighborsClassifier metrics::::::::::::
accuracy score:::::::::::::: 0.3303370786516854
classification report ::::::::::::::
               precision    recall  f1-score   support

           1       0.48      0.43      0.45       336
           2       0.28      0.09      0.13       195
           3       0.26      0.48      0.34       688
           4       0.41      0.20      0.27       989
           5       0.34      0.37      0.36       907

    accuracy                           0.33      3115
   macro avg       0.35      0.31      0.31      3115
weighted avg       0.36      0.33      0.32      3115

confusion matrix ::::::::::::::
 [[143  20  82  20  71]
 [ 52  17  67  22  37]
 [ 33   7 332  98 218]
 [ 41   8 424 199 317]
 [ 31   8 382 148 338]]


In [188]:
import sklearn
sgd=sklearn.linear_model.SGDClassifier()
sgd.fit(x_train,y_train)
y_pred=sgd.predict(x_test)
print('SGDClassifier metrics::::::::::::')
print('accuracy score::::::::::::::',accuracy_score(y_test,y_pred))
print('classification report ::::::::::::::\n',classification_report(y_test,y_pred))
print('confusion matrix ::::::::::::::\n',confusion_matrix(y_test,y_pred))

SGDClassifier metrics::::::::::::
accuracy score:::::::::::::: 0.39357945425361157
classification report ::::::::::::::
               precision    recall  f1-score   support

           1       0.50      0.77      0.61       336
           2       0.31      0.09      0.14       195
           3       0.26      0.11      0.16       688
           4       0.39      0.45      0.42       989
           5       0.38      0.48      0.42       907

    accuracy                           0.39      3115
   macro avg       0.37      0.38      0.35      3115
weighted avg       0.37      0.39      0.36      3115

confusion matrix ::::::::::::::
 [[259   7  12  34  24]
 [103  17  20  37  18]
 [ 68  10  77 267 266]
 [ 55  18  80 442 394]
 [ 32   3 102 339 431]]


In [189]:
######################################################

In [196]:
import gensim.downloader
print(list(gensim.downloader.info().keys()))

['corpora', 'models']


In [199]:
print(list(gensim.downloader.info()['models']))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [200]:
pretrained_word2vec = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [205]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import pad_sequence

AttributeError: module 'tensorflow.core.protobuf.struct_pb2' has no attribute 'TypeSpecProto'

In [ ]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['review'])